In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import requests as req
from tqdm import tqdm

from urllib.request import urlretrieve
from os import listdir, mkdir, path as os_path
from time import sleep

In [32]:
country = 'belgium'
response = req.get(f'https://www.xeno-canto.org/api/2/recordings?query=cnt:{country}')
js = response.json()
js

{'numRecordings': '8211',
 'numSpecies': '270',
 'page': 1,
 'numPages': 17,
 'recordings': [{'id': '544454',
   'gen': 'Oenanthe',
   'sp': 'seebohmi',
   'ssp': '',
   'group': 'birds',
   'en': 'Atlas Wheatear',
   'rec': 'Hans Matheve',
   'cnt': 'Belgium',
   'loc': 'Templeuve, Hainaut',
   'lat': '50.6422',
   'lng': '3.2527',
   'alt': '40',
   'type': 'song',
   'sex': 'male',
   'stage': '',
   'method': 'field recording',
   'url': '//xeno-canto.org/544454',
   'file': 'https://xeno-canto.org/544454/download',
   'file-name': "XC544454-Seebohm's Wheatear (song, BE, Templeuve, 7may2019, 1).mp3",
   'sono': {'small': '//xeno-canto.org/sounds/uploaded/VROUALCTKG/ffts/XC544454-small.png',
    'med': '//xeno-canto.org/sounds/uploaded/VROUALCTKG/ffts/XC544454-med.png',
    'large': '//xeno-canto.org/sounds/uploaded/VROUALCTKG/ffts/XC544454-large.png',
    'full': '//xeno-canto.org/sounds/uploaded/VROUALCTKG/ffts/XC544454-full.png'},
   'osci': {'small': '//xeno-canto.org/sounds/upl

In [2]:
def xeno_scrap(js):
    df_ = pd.DataFrame()
    for n_page in tqdm(range(1, js["numPages"]+1)):
        response = req.get(f'https://www.xeno-canto.org/api/2/recordings?query=cnt:{country}&page={n_page}')
        page_json = response.json()
        ids, files, file_names, ens, lengths, gens, sps, song, q = [], [], [], [], [], [], [], [], []
        for recording in page_json["recordings"]:
            ids.append(recording["id"])
            files.append(recording["file"])
            file_names.append(recording["file-name"])
            ens.append(recording["en"])
            lengths.append(recording["length"])
            gens.append(recording["gen"])
            sps.append(recording["sp"])
            song.append(recording["type"])
            q.append(recording["q"])

        df_ = df_.append(pd.DataFrame.from_records({'id': ids, "file": files, "file-name": file_names, "en": ens, "gen": gens, "sp": sps, "length": lengths, "type": song, "quality": q}))
    return (df_)

countries = ['belgium', 'france', 'germany']
df = pd.DataFrame()

for country in countries:
    response = req.get(f'https://www.xeno-canto.org/api/2/recordings?query=cnt:{country}')
    js = response.json()
    df = df.append(xeno_scrap(js))
df.to_csv("birds.csv", index=False)

  0%|          | 0/17 [00:00<?, ?it/s]C:\Users\Charles Belpois\AppData\Local\Temp\ipykernel_19912\2088668814.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_ = df_.append(pd.DataFrame.from_records({'id': ids, "file": files, "file-name": file_names, "en": ens, "gen": gens, "sp": sps, "length": lengths, "type": song, "quality": q}))
  6%|▌         | 1/17 [00:01<00:17,  1.12s/it]C:\Users\Charles Belpois\AppData\Local\Temp\ipykernel_19912\2088668814.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_ = df_.append(pd.DataFrame.from_records({'id': ids, "file": files, "file-name": file_names, "en": ens, "gen": gens, "sp": sps, "length": lengths, "type": song, "quality": q}))
 12%|█▏        | 2/17 [00:02<00:19,  1.27s/it]C:\Users\Charles Belpois\AppData\Local\Temp\ipykernel_19912\2088668814.py:18: FutureWa

In [3]:
df

,en,file,file-name,gen,id,length,quality,sp,type
0,Atlas Wheatear,https://xeno-canto.org/544454/download,"XC544454-Seebohm's Wheatear (song, BE, Templeu...",Oenanthe,544454,1:08,B,seebohmi,song
1,Fulvous Whistling Duck,https://xeno-canto.org/539628/download,XC539628-Rosse fluiteend 191024 Kalmthoutse he...,Dendrocygna,539628,0:08,C,bicolor,flight call
2,Canada Goose,https://xeno-canto.org/785499/download,XC785499-Bernache_du_Canada_2022-04-14_10-10-1...,Branta,785499,0:53,A,canadensis,call
3,Canada Goose,https://xeno-canto.org/785498/download,XC785498-Bernache_du_Canada_2022-04-14_10-09-3...,Branta,785498,0:09,A,canadensis,call
4,Canada Goose,https://xeno-canto.org/780679/download,XC780679-19-09-22-(bernache-du-Canada-+-battem...,Branta,780679,0:14,A,canadensis,flight call
...,...,...,...,...,...,...,...,...,...
416,Identity unknown,https://xeno-canto.org/277396/download,XC277396-Unbekannt.mp3,Mystery,277396,0:01,E,mystery,call
417,Identity unknown,https://xeno-canto.org/256271/download,XC256271-Sprachmemo192.mp3,Mystery,256271,0:26,E,mystery,song
418,Identity unknown,https://xeno-canto.org/197054/download,XC197054-30469_119 - 4 x mp3.mp3,Mystery,197054,0:02,E,mystery,?
419,Identity unknown,https://xeno-canto.org/179072/download,XC179072-4666-01-3x mp3.mp3,Mystery,179072,0:03,E,mystery,?


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 83220 entries, 0 to 420
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   en         83220 non-null  object
 1   file       83220 non-null  object
 2   file-name  83220 non-null  object
 3   gen        83220 non-null  object
 4   id         83220 non-null  object
 5   length     83220 non-null  object
 6   quality    83220 non-null  object
 7   sp         83220 non-null  object
 8   type       83220 non-null  object
dtypes: object(9)
memory usage: 6.3+ MB


In [5]:
data = df.copy()

In [6]:
data = data[data['en'] != 'Identity unknown']
data = data[data['en'] != 'Soundscape']

In [7]:
data = data[~((data['length'].str[2:] < '10') & (data['length'].str[0:1] == '0'))]

In [8]:
data = data[(data['length'].str[0:1] < '3')]

In [9]:
data

,en,file,file-name,gen,id,length,quality,sp,type
0,Atlas Wheatear,https://xeno-canto.org/544454/download,"XC544454-Seebohm's Wheatear (song, BE, Templeu...",Oenanthe,544454,1:08,B,seebohmi,song
2,Canada Goose,https://xeno-canto.org/785499/download,XC785499-Bernache_du_Canada_2022-04-14_10-10-1...,Branta,785499,0:53,A,canadensis,call
4,Canada Goose,https://xeno-canto.org/780679/download,XC780679-19-09-22-(bernache-du-Canada-+-battem...,Branta,780679,0:14,A,canadensis,flight call
6,Canada Goose,https://xeno-canto.org/627972/download,XC627972-Grote Canadese Gans 20210228 1955 1a.mp3,Branta,627972,0:40,A,canadensis,"alarm call, call, flight call"
7,Canada Goose,https://xeno-canto.org/555675/download,XC555675-Grote Canadese Gans 2020-05-08 2215a.mp3,Branta,555675,0:19,A,canadensis,call
...,...,...,...,...,...,...,...,...,...
338,Nathusius's Pipistrelle,https://xeno-canto.org/816598/download,XC816598-EPTNIL_20230714_230027.wav,Pipistrellus,816598,0:16,A,nathusii,echolocation
341,Nathusius's Pipistrelle,https://xeno-canto.org/811545/download,XC811545-PIPNAT_20230624_231653.wav,Pipistrellus,811545,0:15,A,nathusii,echolocation
342,Common Pipistrelle,https://xeno-canto.org/816644/download,XC816644-PIPPIP_20230714_234334.wav,Pipistrellus,816644,0:12,A,pipistrellus,echolocation
344,Common Pipistrelle,https://xeno-canto.org/821432/download,XC821432-230802-Pipistrellus-pipistrellus-Gesa...,Pipistrellus,821432,0:22,C,pipistrellus,song


In [10]:
data = data[data.quality < 'B']
data

,en,file,file-name,gen,id,length,quality,sp,type
2,Canada Goose,https://xeno-canto.org/785499/download,XC785499-Bernache_du_Canada_2022-04-14_10-10-1...,Branta,785499,0:53,A,canadensis,call
4,Canada Goose,https://xeno-canto.org/780679/download,XC780679-19-09-22-(bernache-du-Canada-+-battem...,Branta,780679,0:14,A,canadensis,flight call
6,Canada Goose,https://xeno-canto.org/627972/download,XC627972-Grote Canadese Gans 20210228 1955 1a.mp3,Branta,627972,0:40,A,canadensis,"alarm call, call, flight call"
7,Canada Goose,https://xeno-canto.org/555675/download,XC555675-Grote Canadese Gans 2020-05-08 2215a.mp3,Branta,555675,0:19,A,canadensis,call
8,Canada Goose,https://xeno-canto.org/554986/download,XC554986-Grote Canadese Gans 2020-05-07 0650a.mp3,Branta,554986,0:55,A,canadensis,"alarm call, flight call"
...,...,...,...,...,...,...,...,...,...
333,Eurasian Serotine,https://xeno-canto.org/809282/download,XC809282-EPTSER_20230616_230149.wav,Eptesicus,809282,0:16,A,serotinus,echolocation
336,Common Noctule,https://xeno-canto.org/818305/download,XC818305-NoID_20230722_223542XC.wav,Nyctalus,818305,0:12,A,noctula,echolocation
338,Nathusius's Pipistrelle,https://xeno-canto.org/816598/download,XC816598-EPTNIL_20230714_230027.wav,Pipistrellus,816598,0:16,A,nathusii,echolocation
341,Nathusius's Pipistrelle,https://xeno-canto.org/811545/download,XC811545-PIPNAT_20230624_231653.wav,Pipistrellus,811545,0:15,A,nathusii,echolocation


In [11]:
data = data[data.type.apply(lambda x: x in 'song')]

In [12]:
data = data[data.type != '']

In [13]:
data

,en,file,file-name,gen,id,length,quality,sp,type
247,Gadwall,https://xeno-canto.org/796746/download,XC796746-Canard-chipeau230204_0819.wav,Mareca,796746,0:28,A,strepera,song
392,Common Pochard,https://xeno-canto.org/770645/download,"XC770645-Fulligule-milouin-(chant,-parade,-Hol...",Aythya,770645,1:27,A,ferina,song
419,Grey Partridge,https://xeno-canto.org/531875/download,"XC531875-Grey Partridge (call, BE, UItkerkse P...",Perdix,531875,0:22,A,perdix,song
421,Grey Partridge,https://xeno-canto.org/280951/download,XC280951-059 3 Patrijs 3 Zang.mp3,Perdix,280951,0:13,A,perdix,song
422,Grey Partridge,https://xeno-canto.org/280950/download,XC280950-059 2 Patrijs 2 Zang.mp3,Perdix,280950,1:30,A,perdix,song
...,...,...,...,...,...,...,...,...,...
126,Tree Cricket,https://xeno-canto.org/590790/download,"XC590790-200921_2700 2008 Weinhähnchen. D, NW,...",Oecanthus,590790,0:14,A,pellucens,song
130,Tree Cricket,https://xeno-canto.org/496756/download,"XC496756-190910 2125 Weinhähnchen D, SU, E Nie...",Oecanthus,496756,0:24,A,pellucens,song
131,Tree Cricket,https://xeno-canto.org/489002/download,"XC489002-190726_1482 2300. Weinhähnchen. D, NW...",Oecanthus,489002,0:50,A,pellucens,song
138,Bog Bush-Cricket,https://xeno-canto.org/446561/download,XC446561-Metrioptera brachyptera.mp3,Metrioptera,446561,0:31,A,brachyptera,song


In [14]:

species = data['en'].value_counts()
species.head(25)

Common Blackbird         416
European Robin           316
Eurasian Blackcap        288
Great Tit                254
Common Chaffinch         237
Song Thrush              231
Common Nightingale       222
Common Whitethroat       214
Eurasian Wren            212
Common Chiffchaff        184
Garden Warbler           170
Eurasian Reed Warbler    154
Willow Warbler           133
Marsh Warbler            123
Melodious Warbler        119
Black Redstart           118
Dunnock                  116
Eurasian Blue Tit        110
Tree Pipit               107
Common Redstart          105
Sedge Warbler            104
Coal Tit                 101
Corn Bunting              99
Common Reed Bunting       98
Great Reed Warbler        97
Name: en, dtype: int64

In [79]:
# large_sp = species[species>=300].index
large_sp = species.head(5).index
data = data[data["en"].isin(large_sp)]
large_sp

Index(['Eurasian Blackcap', 'Common Blackbird', 'European Robin',
       'Song Thrush', 'Great Tit'],
      dtype='object')

In [15]:
large_sp = ['Eurasian Blackcap', 'Common Blackbird', 'European Robin', 'Great Tit', 'Song Thrush']
data = data[data["en"].isin(large_sp)]

In [16]:
data

,en,file,file-name,gen,id,length,quality,sp,type
273,Great Tit,https://xeno-canto.org/820769/download,XC820769-Mesange-charbonniere-230523_1900_1SA_...,Parus,820769,0:20,A,major,song
274,Great Tit,https://xeno-canto.org/820768/download,XC820768-Mesange-charbonniere-230429_1800_1SA_...,Parus,820768,0:36,A,major,song
276,Great Tit,https://xeno-canto.org/798050/download,XC798050-Koolmees-20220321-1600-1a.mp3,Parus,798050,1:28,A,major,song
277,Great Tit,https://xeno-canto.org/798049/download,XC798049-Koolmees-20220321-1225-1a.mp3,Parus,798049,2:22,A,major,song
279,Great Tit,https://xeno-canto.org/788098/download,"XC788098-24-03-23-(mésange-charbonnière,-chant...",Parus,788098,0:33,A,major,song
...,...,...,...,...,...,...,...,...,...
291,European Robin,https://xeno-canto.org/90788/download,2011_12_01_Rotkehlchen_01-35.mp3,Erithacus,90788,2:40,A,rubecula,song
292,European Robin,https://xeno-canto.org/79054/download,RotkehlchWippenb23-5-11~7-00.mp3,Erithacus,79054,1:20,A,rubecula,song
293,European Robin,https://xeno-canto.org/75162/download,Rotk1OberhPanoF6-4-11_17-47xc.mp3,Erithacus,75162,12:11,A,rubecula,song
295,European Robin,https://xeno-canto.org/72638/download,Rotk2LBDreieck12-4-8frueh.mp3,Erithacus,72638,1:22,A,rubecula,song


In [18]:
great_tit = data[data['en'] == "Great Tit"][:230]
eur_robin = data[data['en'] == "European Robin"][:230]
eur_black = data[data['en'] == "Eurasian Blackcap"][:230]
song_t = data[data['en'] == "Song Thrush"][:230]
com_bb = data[data['en'] == "Common Blackbird"][:230]

In [19]:
df2 = pd.DataFrame() 
df2 = df2.append([great_tit, eur_robin, eur_black, song_t, com_bb])

C:\Users\Charles Belpois\AppData\Local\Temp\ipykernel_19912\3460605587.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append([great_tit, eur_robin, eur_black, song_t, com_bb])


In [20]:
species = df2['en'].value_counts().head(5)
print(f"{species.size} different species and a total of {species.values.sum()} samples")
species

5 different species and a total of 1150 samples


Great Tit            230
European Robin       230
Eurasian Blackcap    230
Song Thrush          230
Common Blackbird     230
Name: en, dtype: int64

In [21]:
df2.to_csv("cleaned_birds.csv", index=False)
audio_dir = 'recordings/'
if not os_path.isdir(audio_dir):
    mkdir(audio_dir)

In [25]:
nots = []

for row in tqdm(df2.iterrows(), total=df2.shape[0]):
    url = row[1]["file"]
    f_name = str(row[1]["id"]) + row[1]["file-name"][-4:].lower()
    print(audio_dir + f_name)
    try:
        urlretrieve(url, filename=audio_dir + f_name)
    except Exception as e:
        print("\nRetrying:", url)
        print(e) 
        sleep(60)
        try:
            urlretrieve(url, filename=audio_dir + f_name)
        except Exception as ee:
            print("Not downloaded|", f_name)
            print(ee)
            nots.append(row[1]["id"])
            pass

if len(nots) > 0:
    with open('not_downloaded.txt', 'w') as f:
        for item in nots:
            f.write(str(item) + '\n')
    print(str(nots))
else:
    print('All files were successfully downloaded!')

  0%|          | 0/1150 [00:00<?, ?it/s]

recordings/820769.mp3


  0%|          | 1/1150 [00:00<07:25,  2.58it/s]

recordings/820768.mp3


  0%|          | 2/1150 [00:00<08:38,  2.21it/s]

...

recordings/434474.mp3


100%|█████████▉| 1149/1150 [24:31<00:01,  1.47s/it]

recordings/434471.mp3


100%|██████████| 1150/1150 [24:32<00:00,  1.28s/it]

All files were successfully downloaded!
